# Ziel
Wir wollen mittels der Q-Matrix, die wir mit Q-Learning erzeugen ein neuronales Netz trainieren um eine KI zu erschaffen, die besser Tic Tac Toe spielen kann als die bisherige.

# erste Aufgaben
Wir wollen ein Neuronales Netz trainieren. Dafür müssen wir es zuerst erzeugen, wofür wir die Keras Bibliothek nutzen werden.

Zum Trainieren müssen wir außerdem die Trainingsdaten in ein geeignetes Format bringen.

Also verfolgen wir die folgenden Schritte:

  1. Bereite die Trainingsdaten vor.
  2. Erstelle das Neuronales Netz.
  3. Trainiere das Netzwerk mit diesen Trainingsdaten.
  4. Teste das Netzwerk.
  5. Speichere das Netzwerk um es nicht jedes Mal neu trainieren zu müssen.   

# 1. Traingsdaten vorbereiten

Wir werden das neuronale Netz später mit der `.fit()` Methode von Keras trainieren. Dazu müssen die Daten folgendes Format haben:

- ein array mit Inputwerten im Bereich $[0,1]$ pro Input Node
- ein array mit Zielwerten (Outputwerten), ebenfalls im Bereich $[0,1]$

## unsere bisherigen Daten
Bisher sind unsere Trainingsdaten in einem Dictionary in folgender Form gespeichert

**Keys:**
- Paare (state, action), wobei

    - state ein Tupel aus 9 Integern 0/1/2 ist.
    - action ein einzelner Integer von 0 bis 8 ist.

**Values:**
- Floats, deren Bereich von den Trainingsdaten abhängt.

    - minimaler Wert ist `reward_dict["loss"]`
    - maximaler Wert ist `reward_dict["win"]`

    ...zumindest wenn dies die Extremwerte der Rewards sind, was aber logischerweise der Fall sein sollte

## was wir für Daten wollen
Das Neuronale Netzwerk soll die Q-Funktion approximieren.
Nun haben wir zwei Möglichkeiten, welche Funktion das Netzwerk approximieren soll:

### #1 Eingabewerte nur Zustand
$NN: \mathcal{S} \to \mathbb{R}^{|\mathcal{A}|}$ das heißt das Netzwerk bekommt ein Zustand als Eingabewert und gibt für alle Aktionen, die in dem Zustand möglich sind, eine Wertung aus.

Die Anzahl der möglichen Aktionen ist aber abhängig vom Zustand. Das heißt wir bräuchten immer 9 Ausgabewerte (für jede Aktion einen), aber das Netzwerk müsste zusätzlich lernen, welche Aktionen erlaubt sind.

Alternativ könnte man hier auch einen einzelnen Ausgabewert als die möglichen Aktionen interpretieren. Also durch eine Zustandsabhängige Funktion $f_S:[0,1] \to A\subset \{0,...,8\}$.

### #2 Eingabewerte Zustands-Aktions Paare
$NN: \mathcal{S} \to \mathbb{R}$ Hier würde das Netzwerk zusätzlich zum Zustand des Feldes noch eine Aktion als Input bekommen.

Der Ausgabewert ist dann eine einzige Zahl, die den Wert dieser Aktion repräsentiert.

## Pro und Cons
Variante 2 erfordert es für jede mögliche Aktion den Wert zu berechnen bevor ein Zug gewählt werden kann. Variante 1 verkürzt diesen Prozess indem einfach nur der Maximale Output-Wert genommen wird.

Dieser Unterschied kann durch Wahl der Größe und Komplexität des Netzwerks ausgeglichen werden. Da das Netzwerk in Variante 2 nicht lernen muss, welche Züge überhaupt erlaubt sind, sollte das Training damit aber schneller gehen.

Sicherlich gibt es noch viele andere Möglichkeiten der Implementierung, aber ich werde hier Variante 2 verwenden.

Die Wahl von Variante 2 erlaubt außerdem einfaches Vorbereiten der Daten.

Wir generieren die input Daten also durch

In [1]:
import numpy as np
def prepare_data(Q_table):
    """
    prepare the data given in a Q-table for training the neural network
    """
    training_inputs = []
    training_outputs = []
    for state_action, value in Q_table.items():
        # input_data   =      state_info       +    action_info
        training_inputs.append( list(state_action[0]) + [state_action[1]] )
        training_outputs.append ( value )
    return np.array(training_inputs), np.array(training_outputs)

# 2. Erstelle das Netzwerk (`model`)

In [5]:
# from tensorflow.keras import layers
import tensorflow.keras as keras

def create_ttt_network():
    model = keras.Sequential()
    model.add( keras.Input(shape=(10,)) ) # input layer - 10 Nodes
    model.add( keras.layers.Dense(20) )   # hidden layer 1 - 20 Nodes
    model.add( keras.layers.Dense(10) )   # hidden layer 2 - 10 Nodes

    model.compile()
    return model

ModuleNotFoundError: No module named 'tensorflow.keras'

# 3. Trainiere das Netzwerk mit `.fit()`

In [6]:
def train_network(model, samples, labels, epochs=50, batch_size=32):
    history = model.fit(samples, labels, epochs=epochs, batch_size=batch_size)
    return history

Nun nutzen wir die oben definierten Methoden

In [5]:
def import_Q_table(filename="Q_table.txt"):
    """
    import Q_table as dictionary:
    keys are state-action pairs as a tuple of a tuple (9 integers: 0/1/2) and an integer (0-8)
    values are the corresponding Q-values

    Example:
        Q_table[((0,0,0,0,1,0,0,0,0),2)] -> 0.3
    """
    Q_table = dict()
    with open(filename, "r") as file:
        for line in file.readlines():
            if line == "Q_table = {\n" or line == "}":
                continue
            state_action, value = line[:-2].split(":")
            state = tuple([int(x.strip(" ")) for x in state_action[2:-5].split(",")])
            action = int(state_action[-2])
            Q_table[(state, action)] = float(value)
    return Q_table

In [ ]:
# generate data
Q_table = import_Q_table(filename="Q_table.txt")
# prepare data
inputs, outputs = prepare_data(Q_table)
# create model
model = create_ttt_network()
# train network with given data
history = train_network(model, inputs, outputs, epochs=30, batch_size=32)

